In [359]:
import numpy as np
import pandas as pd
import lightgbm as lgb

!pip install optuna
import optuna
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [360]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # f1_score, recall_score
import lightgbm as lgb

**1. 학습 데이터 생성**

In [361]:
# 데이터셋 가져오기

data = datasets.fetch_covtype()
X = data.data
y = data.target

In [362]:
# 랜덤한 20,000 개의 데이터를 학습

num_samples = 20000
np.random.seed(2025)
indices = np.random.choice(X.shape[0], num_samples, replace=False)

X = X[indices]
y = y[indices]

In [363]:
# 데이터 확인하기

X = pd.DataFrame(X, columns=data.feature_names)

In [364]:
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,3197.0,229.0,15.0,547.0,88.0,450.0,196.0,253.0,191.0,2960.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2483.0,21.0,8.0,218.0,40.0,150.0,215.0,223.0,145.0,1771.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2856.0,63.0,7.0,150.0,8.0,1271.0,227.0,225.0,132.0,2311.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3129.0,100.0,19.0,339.0,68.0,1693.0,248.0,211.0,84.0,2755.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2607.0,69.0,15.0,90.0,-18.0,1020.0,234.0,210.0,103.0,765.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2890.0,115.0,21.0,0.0,0.0,2755.0,250.0,215.0,83.0,1760.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,2191.0,328.0,22.0,277.0,127.0,598.0,162.0,209.0,184.0,484.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,2805.0,341.0,10.0,42.0,8.0,1395.0,199.0,225.0,165.0,2594.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,2964.0,156.0,13.0,216.0,50.0,1615.0,235.0,241.0,134.0,2719.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [365]:
y = pd.DataFrame(y, columns=['target'])

In [366]:
y

,target
0,2
1,3
2,2
3,2
4,2
...,...
19995,1
19996,3
19997,2
19998,2


In [367]:
# Class 분포 확인

y.value_counts()

,count
target,
2,9652
1,7479
3,1170
7,707
6,592
5,307
4,93


In [368]:
# train, test_hpo, test_new 데이터셋으로 Split
# overfitting 여부를 판단하려면 y의 class 분포를 일치시키는 것이 비교적 좋음

from sklearn.model_selection import train_test_split

# [ Entire Dataset ] ---> [ Train ] / [ Test_hpo ] / [ Test_new ]
X_train, X_test_new, y_train, y_test_new =\
    train_test_split(X, y,
                     test_size=0.1,
                     stratify=y,  # overfitting 여부 판단에 도움
                     random_state=2027)

X_train_, X_test_hpo, y_train_, y_test_hpo =\
    train_test_split(X_train, y_train,
                     test_size=0.1,
                     stratify=y_train,  # overfitting 여부 판단에 도움
                     random_state=2027)

In [369]:
# train data 확인

X_train_

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
7448,2094.0,8.0,37.0,212.0,144.0,234.0,152.0,141.0,105.0,190.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8720,2925.0,129.0,9.0,256.0,21.0,3865.0,236.0,235.0,129.0,1595.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12582,3310.0,211.0,11.0,85.0,4.0,2597.0,211.0,251.0,175.0,543.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9579,2859.0,355.0,17.0,42.0,7.0,3966.0,190.0,208.0,154.0,2821.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8198,2559.0,121.0,4.0,589.0,56.0,1090.0,227.0,236.0,144.0,1382.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12639,3187.0,98.0,12.0,175.0,24.0,824.0,240.0,223.0,111.0,2711.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1894,3277.0,29.0,12.0,67.0,37.0,5280.0,216.0,213.0,131.0,124.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14388,3039.0,108.0,26.0,300.0,103.0,1640.0,253.0,199.0,57.0,2663.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18974,3039.0,72.0,17.0,95.0,6.0,4657.0,237.0,204.0,92.0,1018.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [370]:
y_train_

,target
7448,3
8720,2
12582,7
9579,2
8198,3
...,...
12639,2
1894,1
14388,1
18974,2


In [371]:
y_train_count = y_train_.value_counts()
y_train_normalized = y_train_.value_counts(normalize=True)

y_train_distrib = pd.DataFrame({'count': y_train_count, 'percentage': 100 * y_train_normalized})
y_train_distrib

,count,percentage
target,,
2,7818,48.259259
1,6058,37.395062
3,948,5.851852
7,572,3.530864
6,480,2.962963
5,248,1.530864
4,76,0.469136


In [372]:
# test_hpo 데이터 확인 (HPO 성능 최적화를 위한 metric 값 도출용)

X_test_hpo

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
15411,2952.0,128.0,15.0,268.0,3.0,4221.0,244.0,229.0,110.0,5721.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1394,2698.0,25.0,15.0,30.0,3.0,2813.0,211.0,206.0,128.0,1022.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5071,3185.0,117.0,28.0,228.0,72.0,840.0,254.0,200.0,53.0,1683.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19472,2352.0,125.0,15.0,67.0,15.0,1436.0,244.0,228.0,109.0,433.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18191,3025.0,155.0,15.0,127.0,14.0,1598.0,236.0,240.0,130.0,2388.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,2573.0,170.0,16.0,153.0,34.0,1207.0,230.0,245.0,142.0,1818.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4099,3119.0,77.0,6.0,150.0,6.0,3887.0,228.0,228.0,134.0,2920.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
918,3385.0,112.0,8.0,285.0,46.0,1806.0,234.0,232.0,130.0,3113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15268,2959.0,91.0,19.0,67.0,-17.0,395.0,245.0,207.0,84.0,1271.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [373]:
y_test_hpo

,target
15411,2
1394,2
5071,1
19472,3
18191,1
...,...
1513,3
4099,1
918,1
15268,1


In [374]:
y_test_hpo_count = y_test_hpo.value_counts()
y_test_hpo_normalized = y_test_hpo.value_counts(normalize=True)

y_test_hpo_distrib = pd.DataFrame({'count': y_test_hpo_count, 'percentage': 100 * y_test_hpo_normalized})
y_test_hpo_distrib

,count,percentage
target,,
2,869,48.277778
1,673,37.388889
3,105,5.833333
7,64,3.555556
6,53,2.944444
5,28,1.555556
4,8,0.444444


In [375]:
# test_new 데이터 확인 (HPO 완료한 후 최적의 모델을 테스트할 새로운 데이터)

X_test_new

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
7311,2084.0,261.0,39.0,90.0,51.0,400.0,100.0,226.0,247.0,537.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14308,2846.0,54.0,8.0,258.0,30.0,3291.0,225.0,222.0,130.0,2941.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13555,2742.0,36.0,15.0,417.0,-5.0,1560.0,218.0,205.0,119.0,2017.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6011,3134.0,140.0,25.0,510.0,57.0,999.0,248.0,223.0,87.0,2287.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16103,2612.0,332.0,17.0,190.0,55.0,1012.0,177.0,215.0,176.0,655.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,2670.0,64.0,6.0,234.0,9.0,553.0,226.0,228.0,136.0,1132.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5920,2957.0,243.0,19.0,190.0,55.0,2277.0,178.0,251.0,208.0,1430.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18867,3174.0,67.0,31.0,270.0,114.0,1565.0,233.0,162.0,38.0,2557.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6003,3161.0,155.0,18.0,258.0,68.0,192.0,238.0,239.0,124.0,2936.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [376]:
y_test_new

,target
7311,3
14308,2
13555,2
6011,1
16103,2
...,...
1038,2
5920,2
18867,1
6003,1


In [377]:
# y_test_new 의 class 분포가 y_test_old 와 최대한 비슷해야 overfitting 여부 판별에 비교적 도움이 됨

y_test_new_count = y_test_new.value_counts()
y_test_new_normalized = y_test_new.value_counts(normalize=True)

y_test_new_distrib = pd.DataFrame({'count': y_test_new_count, 'percentage': 100 * y_test_new_normalized})
y_test_new_distrib

,count,percentage
target,,
2,965,48.25
1,748,37.40
3,117,5.85
7,71,3.55
6,59,2.95
5,31,1.55
4,9,0.45


In [378]:
# 각 data의 corr-coef 분석

import plotly.figure_factory as ff

# Show corr-coef chart for given dataset
#
# args
# - X         : data to visualize
# - data_name : name for corr-coef chart title

def visualize_corr(X, data_name):
    corr = X.corr()

    fig = ff.create_annotated_heatmap(
                z=corr.round(decimals=2).to_numpy(),
                x=corr.columns.tolist(),
                y=corr.index.tolist(),
                xgap=1,
                ygap=1,
                zmax=1,
                zmin=-1,
                showscale=True,
                colorscale=['#F53', '#BBC', '#57F'],
                hoverongaps=True)

    fig.update_layout(title=f'correlation of {data_name}',
                      width=1500,
                      height=1100)

    fig.show()

In [379]:
# train data 분포 확인 (생략)

# XY_train_ = pd.concat([X_train_, y_train_], axis=1)
# visualize_corr(XY_train_, data_name='train data')

In [380]:
# HPO test data 분포 확인 (생략)

# XY_test_hpo = pd.concat([X_test_hpo, y_test_hpo], axis=1)
# visualize_corr(XY_test_hpo, data_name='test data (for HPO)')

In [381]:
# NEW test data 분포 확인 (생략)

# XY_test_new = pd.concat([X_test_new, y_test_new], axis=1)
# visualize_corr(XY_test_new, data_name='test data (NEW test data)')

In [382]:
# 향후 정확도 측정용

from sklearn.metrics import accuracy_score

**2. HyperOpt 를 이용한 성능 테스트**

In [383]:
# Domain Space 정의

domain_space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(1.0)),
    'max_depth': hp.quniform('max_depth', 1, 100, q=1),
    'num_leaves': hp.quniform('num_leaves', 50, 4000, q=1),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 100, q=1),
    'lambda_l1': hp.uniform('lambda_l1', 0.0, 1.0),
    'lambda_l2': hp.uniform('lambda_l2', 0.0, 5.0)
}

In [384]:
# Objective Function 정의

models = []

def objective_hyperopt(params):
    for param_name in ['max_depth', 'num_leaves', 'min_data_in_leaf']:
        params[param_name] = int(params[param_name])

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=10,  # 일반적으로 너무 작은 값이나, 여기서는 빠른 실험을 위해 사용
        verbosity=-1,
        **params
    )

    # LigthGBM 용 valid set 인 X_valid_lgbm, y_valid_lgbm 로 데이터를 분리
    X_train__, X_valid_lgbm, y_train__, y_valid_lgbm =\
        train_test_split(X_train_, y_train_,
                         test_size=0.15,
                         random_state=2025)

    # X_valid_lgbm, y_valid_lgbm 를 이용하여 LightGBM 모델을 valid
    model.fit(X_train__, y_train__,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 test set인 X_test_hpo, y_test_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_test_hpo)

    accuracy_hpo = accuracy_score(preds_hpo, y_test_hpo)
#    f1_hpo = f1_score(preds_hpo, y_test_hpo)
#    recall_hpo = recall_score(preds_hpo, y_test_hpo)

    acc = f'{(100 * accuracy_hpo):6.2f}'
#    f1 = f'{(100 * f1_hpo):6.2f}'
#    reca = f'{(100 * recall_hpo):6.2f}'

    print(f'[HPO] metrics with {params} : acc={acc}%') #, f1={f1}%, recall={reca}%')

    # 학습한 모델을 최종 테스트를 위해 배열에 저장
    models.append(model)

    # Accuracy 를 최대화 하도록 Loss 반환
    return {"loss": (-1) * accuracy_hpo, "status": STATUS_OK}

In [385]:
# 디버깅 중 불필요한 warning 메시지 미표시

import warnings
warnings.filterwarnings("ignore")

In [386]:
# 하이퍼파라미터 최적화 실시

TRIAL_COUNT = 500

trials = Trials()
best_params_hyperopt = fmin(fn=objective_hyperopt,
                            space=domain_space,
                            algo=tpe.suggest, # tpe = Tree-Structured Parzen Estimator
                            max_evals=TRIAL_COUNT,
                            trials=trials)

[HPO] metrics with {'lambda_l1': 0.37085801390228845, 'lambda_l2': 1.1928907294352442, 'learning_rate': 0.859998289750023, 'max_depth': 83, 'min_data_in_leaf': 20, 'num_leaves': 3163} : acc= 77.89%
[HPO] metrics with {'lambda_l1': 0.24985045976547982, 'lambda_l2': 0.896325742881206, 'learning_rate': 0.00731772040560111, 'max_depth': 84, 'min_data_in_leaf': 16, 'num_leaves': 3735} : acc= 49.67%
[HPO] metrics with {'lambda_l1': 0.09247491842873024, 'lambda_l2': 1.0241600448678412, 'learning_rate': 0.8702577721088897, 'max_depth': 86, 'min_data_in_leaf': 75, 'num_leaves': 66} : acc= 76.22%
[HPO] metrics with {'lambda_l1': 0.0429298253380036, 'lambda_l2': 3.9160943806401214, 'learning_rate': 0.0010286539387261882, 'max_depth': 30, 'min_data_in_leaf': 52, 'num_leaves': 523} : acc= 48.28%
[HPO] metrics with {'lambda_l1': 0.41041701760751914, 'lambda_l2': 4.068837313838413, 'learning_rate': 0.39280569200744325, 'max_depth': 25, 'min_data_in_leaf': 100, 'num_leaves': 73} : acc= 77.50%
[HPO] me

In [387]:
# 각 학습된 모델을 이용하여 New dataset 에 대한 성능 도출

new_dataset_accuracy = []

for i, model in enumerate(models):
    preds_new = model.predict(X_test_new)
    accuracy_new = accuracy_score(preds_new, y_test_new)
    acc = f'{(100 * accuracy_new):6.2f}'

    new_dataset_accuracy.append(100 * accuracy_new)
    print(f'[NEW data] metrics for {i + 1}-th model : acc={acc}%')

[NEW data] metrics for 1-th model : acc= 77.55%
[NEW data] metrics for 2-th model : acc= 49.45%
[NEW data] metrics for 3-th model : acc= 74.90%
[NEW data] metrics for 4-th model : acc= 48.25%
[NEW data] metrics for 5-th model : acc= 76.75%
[NEW data] metrics for 6-th model : acc= 65.80%
[NEW data] metrics for 7-th model : acc= 48.25%
[NEW data] metrics for 8-th model : acc= 48.25%
[NEW data] metrics for 9-th model : acc= 48.25%
[NEW data] metrics for 10-th model : acc= 73.55%
[NEW data] metrics for 11-th model : acc= 66.25%
[NEW data] metrics for 12-th model : acc= 48.25%
[NEW data] metrics for 13-th model : acc= 74.40%
[NEW data] metrics for 14-th model : acc= 80.05%
[NEW data] metrics for 15-th model : acc= 48.25%
[NEW data] metrics for 16-th model : acc= 64.10%
[NEW data] metrics for 17-th model : acc= 67.50%
[NEW data] metrics for 18-th model : acc= 75.15%
[NEW data] metrics for 19-th model : acc= 66.20%
[NEW data] metrics for 20-th model : acc= 48.25%
[NEW data] metrics for 21-th 

In [388]:
# plotly 를 이용하여 성능 비교

WINDOW_SIZE = 30

import plotly.graph_objects as go

trial_nos = np.array(range(TRIAL_COUNT))
hpo_losses = np.array([trial['result']['loss'] for trial in trials.trials])
hpo_accuracy = (-1) * 100.0 * hpo_losses

# 성능 추이 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=hpo_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#F70'},
                         name='HPO test dataset accuracy'))

fig.add_trace(go.Scatter(x=trial_nos,
                         y=new_dataset_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#07F'},
                         name='NEW test dataset accuracy'))

# 성능 추이의 이동평균 표시
hpo_accuracy_ma = np.convolve(hpo_accuracy,
                              np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                              mode='valid')

new_dataset_accuracy_ma = np.convolve(new_dataset_accuracy,
                                      np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                                      mode='valid')

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=hpo_accuracy_ma,
                         mode='lines',
                         line={'color': '#FA0'},
                         name='HPO test dataset accuracy (Moving Average)'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=new_dataset_accuracy_ma,
                         mode='lines',
                         line={'color': '#0AF'},
                         name='NEW test dataset accuracy (Moving Average)'))

# 차트 표시
fig.update_layout(width=1200,
                  height=600,
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy (%)')

fig.show()

In [389]:
# 성능 차이 값의 변화

# 성능 차이 및 그 이동 평균
difference = [float(acc_hpo) - float(acc_new) for acc_hpo, acc_new in zip(hpo_accuracy, new_dataset_accuracy)]
difference_ma = np.convolve(difference, np.ones(WINDOW_SIZE) / WINDOW_SIZE, mode='valid')

# 그래프로 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=difference,
                         mode='markers',
                         marker={'size': 5},
                         name='Accuracy Difference'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=difference_ma,
                         mode='lines',
                         name='Accuracy Difference (Moving Average)'))

fig.update_layout(width=1200,
                  height=600,
                  title='Difference : (HPO dataset ACC) - (NEW dataset ACC)',
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy Difference (%)')

fig.show()

**3. Optuna 를 이용한 성능 테스트**

In [392]:
# Objective Function 정의

models = []

def objective_optuna(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'num_leaves': trial.suggest_int('num_leaves', 50, 4000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 1.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.0, 5.0)
    }

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=10,  # 일반적으로 너무 작은 값이나, 여기서는 빠른 실험을 위해 사용
        verbosity=-1,
        **params
    )

    # LigthGBM 용 valid set 인 X_valid_lgbm, y_valid_lgbm 로 데이터를 분리
    X_train__, X_valid_lgbm, y_train__, y_valid_lgbm =\
        train_test_split(X_train_, y_train_,
                         test_size=0.15,
                         random_state=2025)

    # X_valid_lgbm, y_valid_lgbm 를 이용하여 LightGBM 모델을 valid
    model.fit(X_train__, y_train__,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 test set인 X_test_hpo, y_test_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_test_hpo)

    accuracy_hpo = accuracy_score(preds_hpo, y_test_hpo)
    acc = f'{(100 * accuracy_hpo):6.2f}'

    print(f'[HPO] metrics with {params} : acc={acc}%')

    # 학습한 모델을 최종 테스트를 위해 배열에 저장
    models.append(model)

    # Accuracy 최대화
    return accuracy_hpo

In [394]:
# Optuna warning 미 출력 설정

import logging
optuna.logging.set_verbosity(logging.WARNING)

In [395]:
# 하이퍼파라미터 최적화 실시

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=TRIAL_COUNT)

[HPO] metrics with {'learning_rate': 0.900810876039767, 'max_depth': 14, 'num_leaves': 1174, 'min_data_in_leaf': 47, 'lambda_l1': 0.5598394570528279, 'lambda_l2': 0.5110443809748294} : acc= 76.89%
[HPO] metrics with {'learning_rate': 0.010950383530969648, 'max_depth': 2, 'num_leaves': 3464, 'min_data_in_leaf': 80, 'lambda_l1': 0.6880658750791203, 'lambda_l2': 0.33911886126017254} : acc= 48.28%
[HPO] metrics with {'learning_rate': 0.02292165054624806, 'max_depth': 47, 'num_leaves': 3069, 'min_data_in_leaf': 60, 'lambda_l1': 0.9762139631310837, 'lambda_l2': 0.042479294852333926} : acc= 67.56%
[HPO] metrics with {'learning_rate': 0.051957014089916445, 'max_depth': 67, 'num_leaves': 3310, 'min_data_in_leaf': 4, 'lambda_l1': 0.27922803646779304, 'lambda_l2': 0.717145586516314} : acc= 78.06%
[HPO] metrics with {'learning_rate': 0.2154631462814048, 'max_depth': 49, 'num_leaves': 949, 'min_data_in_leaf': 45, 'lambda_l1': 0.5507823310555864, 'lambda_l2': 2.5132424993728093} : acc= 79.94%
[HPO] 

In [396]:
# 최적의 하이퍼파라미터 확인

best_params_optuna = study.best_params
print(f'Best Parameters : {best_params_optuna}')

Best Parameters : {'learning_rate': 0.2759973064497447, 'max_depth': 42, 'num_leaves': 3393, 'min_data_in_leaf': 3, 'lambda_l1': 0.5840929495125537, 'lambda_l2': 2.9702864733070444}


In [397]:
# 각 학습된 모델을 이용하여 New dataset 에 대한 성능 도출

new_dataset_accuracy = []

for i, model in enumerate(models):
    preds_new = model.predict(X_test_new)
    accuracy_new = accuracy_score(preds_new, y_test_new)
    acc = f'{(100 * accuracy_new):6.2f}'

    new_dataset_accuracy.append(100 * accuracy_new)
    print(f'[NEW data] metrics for {i + 1}-th model : acc={acc}%')

[NEW data] metrics for 1-th model : acc= 74.00%
[NEW data] metrics for 2-th model : acc= 48.25%
[NEW data] metrics for 3-th model : acc= 67.30%
[NEW data] metrics for 4-th model : acc= 77.60%
[NEW data] metrics for 5-th model : acc= 78.10%
[NEW data] metrics for 6-th model : acc= 73.60%
[NEW data] metrics for 7-th model : acc= 76.25%
[NEW data] metrics for 8-th model : acc= 48.25%
[NEW data] metrics for 9-th model : acc= 48.25%
[NEW data] metrics for 10-th model : acc= 73.75%
[NEW data] metrics for 11-th model : acc= 78.55%
[NEW data] metrics for 12-th model : acc= 77.60%
[NEW data] metrics for 13-th model : acc= 78.20%
[NEW data] metrics for 14-th model : acc= 79.75%
[NEW data] metrics for 15-th model : acc= 77.30%
[NEW data] metrics for 16-th model : acc= 80.05%
[NEW data] metrics for 17-th model : acc= 76.05%
[NEW data] metrics for 18-th model : acc= 69.70%
[NEW data] metrics for 19-th model : acc= 76.20%
[NEW data] metrics for 20-th model : acc= 73.15%
[NEW data] metrics for 21-th 

In [399]:
# plotly 를 이용하여 성능 비교

WINDOW_SIZE = 30

import plotly.graph_objects as go

trial_nos = np.array(range(TRIAL_COUNT))
trials_df = study.trials_dataframe()
hpo_accuracy = 100 * trials_df['value'].to_numpy()

# 성능 추이 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=hpo_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#F70'},
                         name='HPO test dataset accuracy'))

fig.add_trace(go.Scatter(x=trial_nos,
                         y=new_dataset_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#07F'},
                         name='NEW test dataset accuracy'))

# 성능 추이의 이동평균 표시
hpo_accuracy_ma = np.convolve(hpo_accuracy,
                              np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                              mode='valid')

new_dataset_accuracy_ma = np.convolve(new_dataset_accuracy,
                                      np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                                      mode='valid')

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=hpo_accuracy_ma,
                         mode='lines',
                         line={'color': '#FA0'},
                         name='HPO test dataset accuracy (Moving Average)'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=new_dataset_accuracy_ma,
                         mode='lines',
                         line={'color': '#0AF'},
                         name='NEW test dataset accuracy (Moving Average)'))

# 차트 표시
fig.update_layout(width=1200,
                  height=600,
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy (%)')

fig.show()

In [400]:
# 성능 차이 값의 변화

# 성능 차이 및 그 이동 평균
difference = [float(acc_hpo) - float(acc_new) for acc_hpo, acc_new in zip(hpo_accuracy, new_dataset_accuracy)]
difference_ma = np.convolve(difference, np.ones(WINDOW_SIZE) / WINDOW_SIZE, mode='valid')

# 그래프로 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=difference,
                         mode='markers',
                         marker={'size': 5},
                         name='Accuracy Difference'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=difference_ma,
                         mode='lines',
                         name='Accuracy Difference (Moving Average)'))

fig.update_layout(width=1200,
                  height=600,
                  title='Difference : (HPO dataset ACC) - (NEW dataset ACC)',
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy Difference (%)')

fig.show()